In [1]:
import tensorflow as tf

# 多进程

In [2]:
# 设置单一变量并初始化为 0
var = tf.Variable(initial_value=0.0)

In [3]:
# 每个进程启动一个会话并初始化全部变量
sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(tf.global_variables_initializer())
sess2.run(tf.global_variables_initializer())

In [4]:
# 每个会话创建单独的执行引擎，各自的变量也互不影响
print("initial value of var in session 1:", sess1.run(var))
print("initial value of var in session 2:", sess2.run(var))

sess1.run(var.assign_add(1.0))
print("increamented var in session 1")

print("value of var in session 1:", sess1.run(var))
print("value of var in session 2:", sess2.run(var))

initial value of var in session 1: 0.0
initial value of var in session 2: 0.0
increamented var in session 1
value of var in session 1: 1.0
value of var in session 2: 0.0


# 分布式

In [5]:
# 对于tensorflow分布式集群，以下代码解释了它的基本原理
c = tf.constant("hello, distibuted tensorflow!")
# 创建一个本地tensorflow集群
server = tf.train.Server.create_local_server()
# 在集群上创建一个会话
sess = tf.Session(server.target)
print(sess.run(c))

b'hello, distibuted tensorflow!'


在以上代码中，我们先通过 tf.train.Server.create_local_server 在本地创建一个只有一台机器的 TensorFlow 集群。然后在集群上生成一个会话，通过该对话，我们可以将创建的计算图运行在 TensorFlow 集群上。虽然这只是一个单机集群，但它基本上反映了 TensorFlow 集群的工作流程。

TensorFlow 集群会通过一系列任务（task）来执行计算图中的运算，一般来说不同的任务会在不同的机器上运行。TensorFlow 集群中的任务也会被聚集为工作（job）。例如在训练深度模型时，一台运行反向传播的机器是一个任务，而所有运行反向传播的集合是一个工作。上面简单的案例只是一个任务的集群，若一个 TensorFlow 集群有多个任务时，我们需要使用 tf.train.ClusterSpec 来指定每一个任务的机器。

使用分布式 TensorFlow 训练深度学习模型一般有两种方式，即 in-graph replication 和 between-graph replication。第一种计算图内的分布式会令所有任务都使用一个 TensorFlow 计算图中的变量，而只是将计算部分分配到不同的服务器上。而另一种计算图间的分布式会在每一个计算服务器上创建一个独立的 TensorFlow 计算图，但不同计算图中的相同参数需要以一种固定的方式存放到同一个参数服务器中。以上大概就是分布式 TensorFlow 的基本概念，随后我们将通过具体的案例与代码加深这一部分的理解。

为了在进程之间共享变量，我们需要将不同的执行引擎连接在一起，并输入分布式张量流。

若使用分布式 TensorFlow，每个进程会运行一个特殊的执行引擎：一个 TensorFlow 服务器。服务器作为集群的一部分链接在一起。（群集中的每个服务器也称为任务。）

第一步是定义集群的规模。我们从最简单的集群开始：即两台服务器（两个任务），它们都在同一台机器上，一个在 2222 端口，一个在 2223 端口。

In [6]:
tasks = ["localhost:2222", "localhost:2223"]

In [7]:
# 每一个任务都与工作相关联
jobs = {"local": tasks}

In [8]:
# 将所有任务定义为一个集群
cluster = tf.train.ClusterSpec(jobs)

我们现在可以启动服务器，指定每个服务器对应为集群定义中的哪个服务器。立即启动各服务器，监听集群设置中指定的端口。

In [9]:
# 指定服务器对应集群中的服务器，启动服务器时集群监听指定端口
server1 = tf.train.Server(cluster, job_name="local", task_index=0)

server2 = tf.train.Server(cluster, job_name="local", task_index=1)

将服务器连接在同一个集群中，我们现在可以体验到分布式 TensorFlow 的强大功能：任何具有相同名称的变量都将在所有服务器之间共享。

最简单的例子是在所有的服务器上运行同一张静态计算图，且每个图只有一个变量：

In [10]:
tf.reset_default_graph()
var = tf.Variable(initial_value=0.0, name="var")
sess1 = tf.Session(server1.target)
sess2 = tf.Session(server2.target)

In [11]:
sess1.run(tf.global_variables_initializer())
#sess2.run(tf.global_variables_initializer()) # 因为只有一个变量，而且该变量是共享的，此处在全局初始化有点多余

print("Initial value of var in session 1:", sess1.run(var))
print("Initial value of var in session 2:", sess2.run(var))

sess1.run(var.assign_add(1.0))
print("Incremented var in session 1")

print("Value of var in session 1:", sess1.run(var))
print("Value of var in session 2:", sess2.run(var))

Initial value of var in session 1: 0.0
Initial value of var in session 2: 0.0
Incremented var in session 1
Value of var in session 1: 1.0
Value of var in session 2: 1.0


## 存放

现在我们可能会想：变量究竟存储在哪个服务器上？又是哪个服务器在运行操作？

按经验来说，变量和操作都默认存储在集群的第一个任务上。

In [12]:
def run_with_location_trace(sess, op):
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    sess.run(op, options=run_options, run_metadata=run_metadata)
    for device in run_metadata.step_stats.dev_stats:
        print(device.device)
        for node in device.node_stats:
            print("\t", node.node_name)

In [13]:
# 在任务 1 中处理变量 var，所有操作将运行在该任务上
run_with_location_trace(sess1, var)

/job:local/replica:0/task:0/cpu:0
	 _SOURCE
/job:local/replica:0/task:0/gpu:0
	 _SOURCE
	 var


In [14]:
run_with_location_trace(sess1, var.assign_add(1.))

/job:local/replica:0/task:0/cpu:0
	 _SOURCE
/job:local/replica:0/task:0/gpu:0
	 _SOURCE
	 AssignAdd_1/value
	 var
	 AssignAdd_1


In [15]:
# 如果在任务 2 中处理变量 var，图节点仍然运行在任务 1 中
run_with_location_trace(sess2, var)

/job:local/replica:0/task:1/cpu:0
	 _SOURCE
/job:local/replica:0/task:0/gpu:0
	 _SOURCE
	 var


In [16]:
# 使用 tf.device 将操作绑定到指定任务中
with tf.device("/job:local/task:0"):
    var1 = tf.Variable(0., name="var1")
with tf.device("/job:local/task:1"):
    var2 = tf.Variable(1., name="var2")
sess1.run(tf.global_variables_initializer())

In [17]:
run_with_location_trace(sess1, var1)

/job:local/replica:0/task:0/cpu:0
	 _SOURCE
/job:local/replica:0/task:0/gpu:0
	 _SOURCE
	 var1


但是 var2 运行在第二个任务上。即使我们尝试使用连接到第一个任务的会话来评估它，它仍然在第二个任务上运行。

In [18]:
run_with_location_trace(sess1, var2)

/job:local/replica:0/task:0/cpu:0
	 _SOURCE
/job:local/replica:0/task:1/gpu:0
	 _SOURCE
	 var2


In [19]:
run_with_location_trace(sess2, var1)

/job:local/replica:0/task:1/cpu:0
	 _SOURCE
/job:local/replica:0/task:0/gpu:0
	 _SOURCE
	 var1


In [20]:
run_with_location_trace(sess2, var2)

/job:local/replica:0/task:1/cpu:0
	 _SOURCE
/job:local/replica:0/task:1/gpu:0
	 _SOURCE
	 var2


## 计算图
分布式 TensorFlow 处理图的过程有几点需要注意。

### 谁构建了这个图？

首先，尽管在整个集群中共享变量值，但图并不会自动共享。

我们用两台服务器创建一个新的集群，然后用显式创建的图设置第一台服务器。

In [21]:
cluster = tf.train.ClusterSpec({"local": ["localhost:2224", "localhost:2225"]})
server1 = tf.train.Server(cluster, job_name="local", task_index=0)
server2 = tf.train.Server(cluster, job_name="local", task_index=1)

In [22]:
graph1 = tf.Graph()
with graph1.as_default():
    var1 = tf.Variable(0., name="var")
sess1 = tf.Session(target=server1.target, graph=graph1)
print(graph1.get_operations())

[<tf.Operation 'var/initial_value' type=Const>, <tf.Operation 'var' type=VariableV2>, <tf.Operation 'var/Assign' type=Assign>, <tf.Operation 'var/read' type=Identity>]


如果我们创建连接到第二台服务器的会话，请注意图不会自动获取镜像。

In [23]:
graph2 = tf.Graph()
sess2 = tf.Session(target=server2.target, graph=graph2)
print(graph2.get_operations())

[]


要访问共享变量，我们必须手动添加一个同名的变量到第二个图中。

In [24]:
with graph2.as_default():
    var2 = tf.Variable(0., name="var")

In [25]:
sess1.run(var1.assign(1.0))
sess2.run(var2)

1.0

关键是：每个服务器负责创建自己的图。

### 所有服务器上的图都必须一样吗？

到目前为止，我们所有的例子都是在两台服务器上运行相同的图。这被称为图内复制（in-graph replication）。

例如，假设我们有一个包含三台服务器的集群。服务器 1 保存共享参数，而服务器 2 和服务器 3 是工作站节点，每个都有本地变量。在图内复制中，每台服务器的图如下所示：

![](../../img/in_graph_replication.png)

图内复制的问题在于每个服务器都必须具有整个图的副本，包括可能只与其他服务器相关的子图。这可能会导致图变得非常大。

另一种方法是图间复制（between-graph replication）。在这里，每个服务器都运行一个只包含共享参数的图，而且任何变量和操作都与单个服务器相关。

![](../../img/between_graph_replication.png)

这种方法缩减了图的大小，因此我们推荐使用图间复制。

## 实践细节
在介绍完整示例之前，有几个实践中遇到的细节问题需要讨论一下。

**如果在所有服务器互联之前尝试在集群上运行某些程序，会发生什么？**

我们再次创建一个双任务集群。

In [26]:
cluster = tf.train.ClusterSpec({"local": ["localhost:2226", "localhost:2227"]})

这一次，让我们在隔离进程中启动每个服务器。（这允许我们随时关闭服务器，以便再次启动它们进行后续的实验。除了关闭启动服务器的进程之外，目前没有其它办法关闭服务器。）

In [27]:
from multiprocessing import Process
from time import sleep

def s1():
    server1 = tf.train.Server(cluster,
                              job_name="local",
                              task_index=0)
    sess1 = tf.Session(server1.target)
    print("server 1: running no-op...")
    sess1.run(tf.no_op())
    print("server 1: no-op run!")
    server1.join() # Block

def s2():
    for i in range(3):
        print("server 2: %d seconds left before connecting..."
              % (3 - i))
        sleep(1.0)
    server2 = tf.train.Server(cluster,
                              job_name="local",
                              task_index=1)
    print("server 2: connected!")
    server2.join() # Block

# daemon=True so that these processes will definitely be killed
# when the notebook restarts
p1 = Process(target=s1, daemon=True)
p2 = Process(target=s2, daemon=True)

服务器 1 即刻加入集群，但服务器 2 在连接之前等待了一会儿。结果如下所示：

In [28]:
p1.start()
p2.start()

server 1: running no-op...


可以看出，每个服务器都试图在集群上运行一个操作，直到所有的服务器都加入。

In [29]:
p1.terminate()
p2.terminate()

## 当服务器脱离集群会怎样？

我们用两台服务器建立一个集群。服务器 1 只是反复尝试和运行位于服务器 1 上的 no-op 操作。服务器 2 将在两秒钟后宕机。


In [30]:
def s1():
    server1 = tf.train.Server(cluster,
                              job_name="local",
                              task_index=0)
    
    with tf.device("/job:local/task:0"):
        no_op = tf.no_op()
        
    sess1 = tf.Session(server1.target)
    for _ in range(6):
        print("Server 1: about to run no-op...", end="")
        sess1.run(no_op)
        print("success!")
        sleep(1.0)

def s2():
    server2 = tf.train.Server(cluster,
                              job_name="local",
                              task_index=1)
    sleep(2.0)
    print("Server 2 dieing...")
    
p1 = Process(target=s1, daemon=True)
p2 = Process(target=s2, daemon=True)

p1.start()
p2.start()

Server 2 dieing...


短期内，只要我们试图运行的操作不在脱离的服务器上，似乎不会出现问题。（我没有测试过长期运行会发生什么。）

如果操作是在脱离的服务器上……

In [31]:
def s1():
    server1 = tf.train.Server(cluster,
                              job_name="local",
                              task_index=0)
    
    # This time, we place the no-op on server 2,
    # which is going to leave
    with tf.device("/job:local/task:1"):
        no_op = tf.no_op()
        
    sess1 = tf.Session(server1.target)
    for _ in range(5):
        print("Server 1: about to run no-op...", end="")
        sess1.run(no_op)
        print("success!")
        sleep(1.0)
    
p1 = Process(target=s1, daemon=True)
p2 = Process(target=s2, daemon=True)

p1.start()
p2.start()

Process Process-5:
Traceback (most recent call last):
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-31-7403e97fe700>", line 4, in s1
    task_index=0)
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/training/server_lib.py", line 145, in __init__
    self._server_def.SerializeToString(), status)
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/contextlib.py", line 88, in __exit__
    next(self.gen)
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/errors_impl.py", line 466, in raise_exception_on_not_ok_status
    pywrap_tensorflow.TF_GetCode(status))
tensorflow.python.framework.errors_impl.UnknownError: Could not start gRPC server


Server 2 dieing...


In [32]:
p1.terminate()
p2.terminate()

In [33]:
p1 = Process(target=s1, daemon=True)
p2 = Process(target=s2, daemon=True)
p1.start()
p2.start()
sleep(3.0)
# At this point, server 1 is blocked, and server 2 is dead.
print("Restarting server 2...")
p2 = Process(target=s2, daemon=True)
p2.start()

Process Process-7:
Traceback (most recent call last):
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-31-7403e97fe700>", line 4, in s1
    task_index=0)
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/training/server_lib.py", line 145, in __init__
    self._server_def.SerializeToString(), status)
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/contextlib.py", line 88, in __exit__
    next(self.gen)
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/errors_impl.py", line 466, in raise_exception_on_not_ok_status
    pywrap_tensorflow.TF_GetCode(status))
tensorflow.python.framework.errors_impl.UnknownError: Could not start gRPC server


Server 2 dieing...
Restarting server 2...
Server 2 dieing...


## 谁负责初始化共享变量？

一种方法是让所有工作站运行 tf.global_variables_initializer()。

但是如果我们想保持代码整洁并且只用一个服务器进行初始化，那么如果有其他服务器在初始化之前尝试使用这些变量，可能会遇到问题。一个解决方案就是让其他工作站等待，直到使用 tf.report_uninitialized_variables 的初始化开始。

In [34]:
def s1():
    server1 = tf.train.Server(cluster,
                              job_name="local",
                              task_index=0)
    var = tf.Variable(0.0, name='var')
    sess1 = tf.Session(server1.target)
    
    print("Server 1: waiting for connection...")
    sess1.run(tf.report_uninitialized_variables())
    while len(sess1.run(tf.report_uninitialized_variables())) > 0:
        print("Server 1: waiting for initialization...")
        sleep(1.0)
    print("Server 1: variables initialized!")

def s2():
    server2 = tf.train.Server(cluster,
                              job_name="local",
                              task_index=1)
    var = tf.Variable(0.0, name='var')
    sess2 = tf.Session(server2.target)
    
    for i in range(3):
        print("Server 2: waiting %d seconds before initializing..."
              % (3 - i))
        sleep(1.0)
    sess2.run(tf.global_variables_initializer())
    
p1 = Process(target=s1, daemon=True)
p2 = Process(target=s2, daemon=True)
p1.start()
p2.start()

Process Process-10:
Traceback (most recent call last):
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-34-c1c49751100d>", line 4, in s1
    task_index=0)
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/training/server_lib.py", line 145, in __init__
    self._server_def.SerializeToString(), status)
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/contextlib.py", line 88, in __exit__
    next(self.gen)
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/errors_impl.py", line 466, in raise_exception_on_not_ok_status
    pywrap_tensorflow.TF_GetCode(status))
tensorflow.python.framework.errors_impl.UnknownError: Could not start gRPC server


Server 2: waiting 3 seconds before initializing...
Server 2: waiting 2 seconds before initializing...
Server 2: waiting 1 seconds before initializing...


In [35]:
p1.terminate()
p2.terminate()

# 示例

让我们把所学的知识融合到最后一个使用多进程的例子中。

我们将创建：

一个存储单个变量 var 的参数服务器。
两个工作站任务（worker task），每个工作站将多次增加变量 var 的值。

我们将让参数服务器多输出几次 var 的值，以便查看其变化。

In [36]:
import tensorflow as tf
from multiprocessing import Process
from time import sleep

cluster = tf.train.ClusterSpec({
    "worker": [
        "localhost:3333",
        "localhost:3334",
    ],
    "ps": [
        "localhost:3335"
    ]
})

def parameter_server():
    with tf.device("/job:ps/task:0"):
        var = tf.Variable(0.0, name='var')

    server = tf.train.Server(cluster,
                             job_name="ps",
                             task_index=0)
    sess = tf.Session(target=server.target)
    
    print("Parameter server: waiting for cluster connection...")
    sess.run(tf.report_uninitialized_variables())
    print("Parameter server: cluster ready!")
    
    print("Parameter server: initializing variables...")
    sess.run(tf.global_variables_initializer())
    print("Parameter server: variables initialized")
    
    for i in range(5):
        val = sess.run(var)
        print("Parameter server: var has value %.1f" % val)
        sleep(1.0)

    print("Parameter server: blocking...")
    server.join()
    

def worker(worker_n):
    with tf.device("/job:ps/task:0"):
        var = tf.Variable(0.0, name='var')
        
    server = tf.train.Server(cluster,
                             job_name="worker",
                             task_index=worker_n)
    sess = tf.Session(target=server.target)
    
    print("Worker %d: waiting for cluster connection..." % worker_n)
    sess.run(tf.report_uninitialized_variables())
    print("Worker %d: cluster ready!" % worker_n)
    
    while sess.run(tf.report_uninitialized_variables()):
        print("Worker %d: waiting for variable initialization..." % worker_n)
        sleep(1.0)
    print("Worker %d: variables initialized" % worker_n)
    
    for i in range(5):
        print("Worker %d: incrementing var" % worker_n)
        sess.run(var.assign_add(1.0))
        sleep(1.0)
    
    print("Worker %d: blocking..." % worker_n)
    server.join()

ps_proc = Process(target=parameter_server, daemon=True)
w1_proc = Process(target=worker, args=(0, ), daemon=True)
w2_proc = Process(target=worker, args=(1, ), daemon=True)

In [37]:
ps_proc.start()

Parameter server: waiting for cluster connection...


In [38]:
w1_proc.start()

Worker 0: waiting for cluster connection...


In [40]:
w2_proc.start()

AssertionError: cannot start a process twice

In [41]:
for proc in [w1_proc, w2_proc, ps_proc]:
    proc.terminate()

# 总结

通过本文，我们了解了：

- 如何将多个 TensorFlow 执行引擎（运行在不同进程或不同机器上）集成为一个集群，以便共享变量。
- 如何为变量或操作指定服务器。
- 图内复制与图间复制。
- 在所有服务器互联之前或在服务器脱离集群之后在集群上运行操作，会发生什么。
- 如何等待变量被集群中的另一个任务初始化。